<a href="https://colab.research.google.com/github/RoseJaisil/Covid19_data_analysis_using_python/blob/master/COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Required Libraries

In [1]:
!pip install catalyst


     |████████████████████████████████| 491kB 18.8MB/s 
     |████████████████████████████████| 645kB 46.2MB/s 
     |████████████████████████████████| 122kB 57.0MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/MyDrive/Class notes/Machine Learning/Project/Covid-CT".zip -d "/content/a"

Archive:  /content/gdrive/MyDrive/Class notes/Machine Learning/Project/Covid-CT.zip
   creating: /content/a/Covid-CT/
  inflating: /content/a/__MACOSX/._Covid-CT  
  inflating: /content/a/Covid-CT/.DS_Store  
  inflating: /content/a/__MACOSX/Covid-CT/._.DS_Store  
   creating: /content/a/Covid-CT/root_dir/
  inflating: /content/a/Covid-CT/root_dir/.DS_Store  
  inflating: /content/a/__MACOSX/Covid-CT/root_dir/._.DS_Store  
   creating: /content/a/Covid-CT/root_dir/baseline methods/
  inflating: /content/a/__MACOSX/Covid-CT/root_dir/._baseline methods  
   creating: /content/a/Covid-CT/root_dir/CT_IMAGES/
  inflating: /content/a/Covid-CT/root_dir/COVID.ipynb  
  inflating: /content/a/__MACOSX/Covid-CT/root_dir/._COVID.ipynb  
  inflating: /content/a/Covid-CT/root_dir/COVID-CT-MetaInfo.xlsx  
  inflating: /content/a/__MACOSX/Covid-CT/root_dir/._COVID-CT-MetaInfo.xlsx  
  inflating: /content/a/Covid-CT/root_dir/NonCOVID-CT-MetaInfo.csv  
  inflating: /content/a/__MACOSX/Covid-CT/root_dir/

In [4]:
import torch
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
from skimage.io import imread, imsave
import skimage

torch.cuda.empty_cache()

## Data Preparation

In [9]:
# Normalization, Image Resizing

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

In [10]:
batchsize=10
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

In [11]:
# Function for median filtering 
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.utils import _pair, _quadruple


class MedianPool2d(nn.Module):
    """ Median pool (usable as median filter when stride=1) module.
    
    Args:
         kernel_size: size of pooling kernel, int or 2-tuple
         stride: pool stride, int or 2-tuple
         padding: pool padding, int or 4-tuple (l, r, t, b) as in pytorch F.pad
         same: override padding and enforce same padding, boolean
    """
    def __init__(self, kernel_size=3, stride=1, padding=0, same=False):
        super(MedianPool2d, self).__init__()
        self.k = _pair(kernel_size)
        self.stride = _pair(stride)
        self.padding = _quadruple(padding)  # convert to l, r, t, b
        self.same = same

    def _padding(self, x):
        if self.same:
            ih, iw = x.size()[2:]
            if ih % self.stride[0] == 0:
                ph = max(self.k[0] - self.stride[0], 0)
            else:
                ph = max(self.k[0] - (ih % self.stride[0]), 0)
            if iw % self.stride[1] == 0:
                pw = max(self.k[1] - self.stride[1], 0)
            else:
                pw = max(self.k[1] - (iw % self.stride[1]), 0)
            pl = pw // 2
            pr = pw - pl
            pt = ph // 2
            pb = ph - pt
            padding = (pl, pr, pt, pb)
        else:
            padding = self.padding
        return padding
    
    def forward(self, x):
        # using existing pytorch functions and tensor ops so that we get autograd, 
        # would likely be more efficient to implement from scratch at C/Cuda level
        x = F.pad(x, self._padding(x), mode='reflect')
        x = x.unfold(2, self.k[0], self.stride[0]).unfold(3, self.k[1], self.stride[1])
        x = x.contiguous().view(x.size()[:4] + (-1,)).median(dim=-1)[0]
        return x

In [12]:
class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample



    
if __name__ == '__main__':
#    trainset = CovidCTDataset(root_dir='CT_IMAGES',
#                              txt_COVID='txtdata/newtxt/trainCT_COVID.txt',
#                              txt_NonCOVID='txtdata/oldtxt/trainCT_NonCOVID.txt',
#                              transform= train_transformer)
#    valset = CovidCTDataset(root_dir='CT_IMAGES',
#                              txt_COVID='txtdata/newtxt/valCT_COVID.txt',
#                              txt_NonCOVID='txtdata/oldtxt/valCT_NonCOVID.txt',
#                              transform= val_transformer)
#    testset = CovidCTDataset(root_dir='CT_IMAGES',
#                              txt_COVID='txtdata/newtxt/testCT_COVID.txt',
#                              txt_NonCOVID='txtdata/oldtxt/testCT_NonCOVID.txt',
#                              transform= val_transformer)

# I have commented the above  to change the file directory
    trainset = CovidCTDataset(root_dir='/content/a/Covid-CT/root_dir/CT_IMAGES', 
                              txt_COVID='/content/a/Covid-CT/root_dir/txtdata/newtxt/trainCT_COVID.txt',
                              txt_NonCOVID='/content/a/Covid-CT/root_dir/txtdata/oldtxt/trainCT_NonCOVID.txt',transform= train_transformer)
    valset = CovidCTDataset(root_dir='/content/a/Covid-CT/root_dir/CT_IMAGES',
                              txt_COVID='/content/a/Covid-CT/root_dir/txtdata/newtxt/valCT_COVID.txt',
                              txt_NonCOVID='/content/a/Covid-CT/root_dir/txtdata/oldtxt/valCT_NonCOVID.txt',
                              transform= val_transformer)
    testset = CovidCTDataset(root_dir='/content/a/Covid-CT/root_dir/CT_IMAGES',
                              txt_COVID='/content/a/Covid-CT/root_dir/txtdata/newtxt/testCT_COVID.txt',
                              txt_NonCOVID='/content/a/Covid-CT/root_dir/txtdata/oldtxt/testCT_NonCOVID.txt',
                              transform= val_transformer)
    print(trainset.__len__())
    print(valset.__len__())
    print(testset.__len__())

    train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=False, shuffle=True)
    val_loader = DataLoader(valset, batch_size=batchsize, drop_last=False, shuffle=False)
    test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)
    

425
118
203


In [14]:
# testing for one image
print(train_loader)

## Train, Validation, Test Functions

In [ ]:
# Training 

alpha = None
device = 'cuda'
def train(optimizer, epoch):
    
    model.train()
    
    train_loss = 0
    train_correct = 0
    
    for batch_index, batch_samples in enumerate(train_loader):
        
        # move data to device
        data, target = batch_samples['img'], batch_samples['label']
#        data = data[:, 0, :, :]
#        data = data[:, None, :, :]
#         data, targets_a, targets_b, lam = mixup_data(data, target.long(), alpha, use_cuda=True)
        
        
        optimizer.zero_grad()
        output = model(data)
        
        criteria = nn.CrossEntropyLoss()
        loss = criteria(output, target.long())
#         loss = mixup_criterion(criteria, output, targets_a, targets_b, lam)
        train_loss += criteria(output, target.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long().view_as(pred)).sum().item()
    
        # Display progress and write to tensorboard
        if batch_index % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))


In [ ]:
# Validation

def val(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    criteria = nn.CrossEntropyLoss()
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        for batch_index, batch_samples in enumerate(val_loader):
            data, target = batch_samples['img'], batch_samples['label']
#            data = data[:, 0, :, :]
#            data = data[:, None, :, :]
            output = model(data)
            
            test_loss += criteria(output, target.long())
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
          
    return targetlist, scorelist, predlist

In [ ]:
# Test

def test(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    criteria = nn.CrossEntropyLoss()
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        for batch_index, batch_samples in enumerate(test_loader):
            data, target = batch_samples['img'], batch_samples['label']
#            data = data[:, 0, :, :]
#            data = data[:, None, :, :]
#             print(target)
            output = model(data)
            
            test_loss += criteria(output, target.long())
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
#             TP += ((pred == 1) & (target.long()[:, 2].view_as(pred).data == 1)).cpu().sum()
#             TN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
# #             # FN    predict 0 label 1
#             FN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 1)).cpu().sum()
# #             # FP    predict 1 label 0
#             FP += ((pred == 1) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
#             print(TP,TN,FN,FP)
            
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
    return targetlist, scorelist, predlist

## ResNet18

In [ ]:
### ResNet18
import torchvision.models as models
model = models.resnet18(pretrained=True)
modelname = 'ResNet18'

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
# train
bs = 10
votenum = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    train(optimizer, epoch)
    
    targetlist, scorelist, predlist = val(epoch)
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        #torch.save(model.state_dict(), "model_backup/{}.pt".format(modelname))  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open('model_result/{}.txt'.format(modelname), 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()


FileNotFoundError: ignored

In [ ]:
# test
bs = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    
    targetlist, scorelist, predlist = test(epoch)
#     print('target',targetlist)
#     print('score',scorelist)
#     print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 
    
    TP = ((predlist == 1) & (targetlist == 1)).sum()
    TN = ((predlist == 0) & (targetlist == 0)).sum()
    FN = ((predlist == 0) & (targetlist == 1)).sum()
    FP = ((predlist == 1) & (targetlist == 0)).sum()

    print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
    print('TP+FP',TP+FP)
    p = TP / (TP + FP)
    print('precision',p)
    p = TP / (TP + FP)
    r = TP / (TP + FN)
    print('recall',r)
    F1 = 2 * r * p / (r + p)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print('F1',F1)
    print('acc',acc)
    AUC = roc_auc_score(targetlist, vote_score)
    print('AUC', AUC)

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        
#         print('vote_pred', vote_pred)
#         print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUC', AUC)
        
        
#         f = open('model_result/{modelname}.txt', 'a+')
#         f.write('precision, recall, F1, acc= \n')
#         f.writelines(str(p))
#         f.writelines('\n')
#         f.writelines(str(r))
#         f.writelines('\n')
#         f.writelines(str(F1))
#         f.writelines('\n')
#         f.writelines(str(acc))
#         f.writelines('\n')
#         f.close()
        
        
        vote_pred = np.zeros((1,testset.__len__()))
        vote_score = np.zeros(testset.__len__())
        print('vote_pred',vote_pred)
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open(f'model_result/test_{modelname}.txt', 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()

TP= 95 TN= 54 FN= 10 FP= 44
TP+FP 139
precision 0.6834532374100719
recall 0.9047619047619048
F1 0.7786885245901638
acc 0.7339901477832512
AUC 0.7470359572400388
TP= 95 TN= 54 FN= 10 FP= 44
TP+FP 139
precision 0.6834532374100719
recall 0.9047619047619048
F1 0.7786885245901638
acc 0.7339901477832512
AUC 0.7470359572400388
TP= 95 TN= 54 FN= 10 FP= 44
TP+FP 139
precision 0.6834532374100719
recall 0.9047619047619048
F1 0.7786885245901638
acc 0.7339901477832512
AUC 0.7470359572400388
TP= 95 TN= 54 FN= 10 FP= 44
TP+FP 139
precision 0.6834532374100719
recall 0.9047619047619048
F1 0.7786885245901638
acc 0.7339901477832512
AUC 0.7470359572400388
TP= 95 TN= 54 FN= 10 FP= 44
TP+FP 139
precision 0.6834532374100719
recall 0.9047619047619048
F1 0.7786885245901638
acc 0.7339901477832512
AUC 0.7470359572400388
TP= 95 TN= 54 FN= 10 FP= 44
TP+FP 139
precision 0.6834532374100719
recall 0.9047619047619048
F1 0.7786885245901638
acc 0.7339901477832512
AUC 0.7470359572400388
TP= 95 TN= 54 FN= 10 FP= 44
TP+FP 

## DenseNet121

In [ ]:
### DenseNet121
import torchvision.models as models
model = models.densenet121(pretrained=True)
modelname = 'Dense121'

In [ ]:
# train
bs = 10
votenum = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    train(optimizer, epoch)
    
    targetlist, scorelist, predlist = val(epoch)
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        #torch.save(model.state_dict(), "model_backup/{}.pt".format(modelname))  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open('model_result/{}.txt'.format(modelname), 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()


Train Epoch: 1 [0/43 (0%)]	Train Loss: 0.869633
Train Epoch: 1 [10/43 (23%)]	Train Loss: 0.763709
Train Epoch: 1 [20/43 (47%)]	Train Loss: 0.406906
Train Epoch: 1 [30/43 (70%)]	Train Loss: 0.249411
Train Epoch: 1 [40/43 (93%)]	Train Loss: 0.209494

Train set: Average loss: 0.5152, Accuracy: 122/425 (29%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [2.63036636e-04 3.84640926e-03 5.34429739e-04 1.31882989e-05
 4.43945173e-05 9.03459113e-06 3.13100638e-04 1.22380897e-03
 3.91588197e-04 1.90104553e-36 7.74898197e-19 2.40039473e-31
 2.31438756e-13 3.26608075e-03 5.18970331e-03 1.47144252e-04
 1.07644082e-04 2.01570927e-10 7.50218751e-04 1.09836238e-03
 1.75540149e-03

Train Epoch: 4 [0/43 (0%)]	Train Loss: 0.012591
Train Epoch: 4 [10/43 (23%)]	Train Loss: 0.013055
Train Epoch: 4 [20/43 (47%)]	Train Loss: 0.074261
Train Epoch: 4 [30/43 (70%)]	Train Loss: 0.069100
Train Epoch: 4 [40/43 (93%)]	Train Loss: 0.006538

Train set: Average loss: 0.0316, Accuracy: 390/425 (92%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [3.52066581e-06 5.73332198e-02 1.40367437e-03 3.71857244e-03
 3.55409761e-03 7.68519158e-07 2.34866571e-02 9.21199622e-04
 1.04750477e-04 1.73904752e-07 2.57165790e-01 8.62952675e-07
 1.31068155e-01 1.75896457e-05 2.22468423e-03 3.20544314e-05
 2.36920523e-03 9.84014332e-06 9.90232220e-04 2.30721649e-04
 4.25905250e-02

Train Epoch: 7 [0/43 (0%)]	Train Loss: 0.033256
Train Epoch: 7 [10/43 (23%)]	Train Loss: 0.002712
Train Epoch: 7 [20/43 (47%)]	Train Loss: 0.003892
Train Epoch: 7 [30/43 (70%)]	Train Loss: 0.017923
Train Epoch: 7 [40/43 (93%)]	Train Loss: 0.002329

Train set: Average loss: 0.0258, Accuracy: 396/425 (93%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [4.75869529e-05 1.39284283e-01 1.31533723e-02 1.99318320e-01
 4.60450873e-02 1.32804969e-04 1.15373373e-01 4.15368676e-02
 4.89296159e-04 5.83656620e-05 7.62714922e-01 8.57391933e-05
 8.95826399e-01 1.04823928e-04 2.40206230e-03 2.23803981e-05
 1.05869293e-03 1.24618993e-04 4.50784923e-04 4.24680859e-03
 4.99290705e-01

Train Epoch: 10 [0/43 (0%)]	Train Loss: 0.000991
Train Epoch: 10 [10/43 (23%)]	Train Loss: 0.008343
Train Epoch: 10 [20/43 (47%)]	Train Loss: 0.005627
Train Epoch: 10 [30/43 (70%)]	Train Loss: 0.000473
Train Epoch: 10 [40/43 (93%)]	Train Loss: 0.001696

Train set: Average loss: 0.0158, Accuracy: 399/425 (94%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [2.72657431e-04 9.04645562e-01 5.02343476e-01 8.14079270e-02
 2.25196662e-03 3.81940736e-05 8.86837542e-01 1.33025527e-01
 1.81482499e-03 1.18698904e-06 5.40698111e-01 2.37678978e-06
 9.10274565e-01 3.36725971e-05 1.29540386e-02 5.44774639e-06
 1.25589140e-04 2.31853290e-03 2.68146559e-03 3.26093705e-03
 4.2758518

In [ ]:
# test
bs = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    
    targetlist, scorelist, predlist = test(epoch)
#     print('target',targetlist)
#     print('score',scorelist)
#     print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 
    
    TP = ((predlist == 1) & (targetlist == 1)).sum()
    TN = ((predlist == 0) & (targetlist == 0)).sum()
    FN = ((predlist == 0) & (targetlist == 1)).sum()
    FP = ((predlist == 1) & (targetlist == 0)).sum()

    print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
    print('TP+FP',TP+FP)
    p = TP / (TP + FP)
    print('precision',p)
    p = TP / (TP + FP)
    r = TP / (TP + FN)
    print('recall',r)
    F1 = 2 * r * p / (r + p)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print('F1',F1)
    print('acc',acc)
    AUC = roc_auc_score(targetlist, vote_score)
    print('AUC', AUC)

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        
#         print('vote_pred', vote_pred)
#         print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUC', AUC)
        
        
#         f = open('model_result/{modelname}.txt', 'a+')
#         f.write('precision, recall, F1, acc= \n')
#         f.writelines(str(p))
#         f.writelines('\n')
#         f.writelines(str(r))
#         f.writelines('\n')
#         f.writelines(str(F1))
#         f.writelines('\n')
#         f.writelines(str(acc))
#         f.writelines('\n')
#         f.close()
        
        
        vote_pred = np.zeros((1,testset.__len__()))
        vote_score = np.zeros(testset.__len__())
        print('vote_pred',vote_pred)
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open(f'model_result/test_{modelname}.txt', 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()

TP= 98 TN= 58 FN= 7 FP= 40
TP+FP 138
precision 0.7101449275362319
recall 0.9333333333333333
F1 0.8065843621399177
acc 0.7684729064039408
AUC 0.8629737609329446
TP= 98 TN= 58 FN= 7 FP= 40
TP+FP 138
precision 0.7101449275362319
recall 0.9333333333333333
F1 0.8065843621399177
acc 0.7684729064039408
AUC 0.8629737609329446
TP= 98 TN= 58 FN= 7 FP= 40
TP+FP 138
precision 0.7101449275362319
recall 0.9333333333333333
F1 0.8065843621399177
acc 0.7684729064039408
AUC 0.8629737609329446
TP= 98 TN= 58 FN= 7 FP= 40
TP+FP 138
precision 0.7101449275362319
recall 0.9333333333333333
F1 0.8065843621399177
acc 0.7684729064039408
AUC 0.8629737609329446
TP= 98 TN= 58 FN= 7 FP= 40
TP+FP 138
precision 0.7101449275362319
recall 0.9333333333333333
F1 0.8065843621399177
acc 0.7684729064039408
AUC 0.8629737609329446
TP= 98 TN= 58 FN= 7 FP= 40
TP+FP 138
precision 0.7101449275362319
recall 0.9333333333333333
F1 0.8065843621399177
acc 0.7684729064039408
AUC 0.8629737609329446
TP= 98 TN= 58 FN= 7 FP= 40
TP+FP 138
pre

## DenseNet169

In [ ]:
### DenseNet169
import torchvision.models as models
model = models.densenet169(pretrained=True)
modelname = 'Dense169'

In [ ]:
# train
bs = 10
votenum = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    train(optimizer, epoch)
    
    targetlist, scorelist, predlist = val(epoch)
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        #torch.save(model.state_dict(), "model_backup/{}.pt".format(modelname))  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open('model_result/{}.txt'.format(modelname), 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()


Train Epoch: 1 [0/43 (0%)]	Train Loss: 0.806527
Train Epoch: 1 [10/43 (23%)]	Train Loss: 0.637306
Train Epoch: 1 [20/43 (47%)]	Train Loss: 0.350420
Train Epoch: 1 [30/43 (70%)]	Train Loss: 0.104904
Train Epoch: 1 [40/43 (93%)]	Train Loss: 0.131919

Train set: Average loss: 0.4396, Accuracy: 174/425 (41%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [2.16312334e-03 1.16907239e-01 5.32385558e-02 3.43045758e-06
 1.72861814e-09 3.36731517e-07 9.98506993e-02 3.08510242e-03
 1.07398151e-12 5.20624003e-29 3.49080855e-12 3.81883893e-08
 1.75240891e-10 4.94399166e-04 4.04749587e-02 1.56492631e-11
 3.59858223e-06 6.70290974e-05 2.82373535e-03 1.48302875e-02
 3.39896232e-02

Train Epoch: 4 [0/43 (0%)]	Train Loss: 0.004024
Train Epoch: 4 [10/43 (23%)]	Train Loss: 0.024802
Train Epoch: 4 [20/43 (47%)]	Train Loss: 0.015114
Train Epoch: 4 [30/43 (70%)]	Train Loss: 0.002838
Train Epoch: 4 [40/43 (93%)]	Train Loss: 0.002261

Train set: Average loss: 0.0227, Accuracy: 396/425 (93%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [6.86533225e-04 4.15410906e-01 4.32047933e-01 7.86269307e-02
 7.02782907e-03 1.35690016e-05 9.19091105e-01 1.93061437e-02
 2.26820175e-05 8.66804063e-01 5.47860742e-01 7.34329402e-01
 4.60559577e-01 9.25725326e-04 4.27898169e-02 7.41380281e-05
 1.04819611e-03 2.76237819e-02 2.21440978e-02 2.04083901e-02
 5.23924939e-02

Train Epoch: 7 [0/43 (0%)]	Train Loss: 0.014917
Train Epoch: 7 [10/43 (23%)]	Train Loss: 0.002385
Train Epoch: 7 [20/43 (47%)]	Train Loss: 0.001791
Train Epoch: 7 [30/43 (70%)]	Train Loss: 0.008968
Train Epoch: 7 [40/43 (93%)]	Train Loss: 0.007714

Train set: Average loss: 0.0143, Accuracy: 409/425 (96%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [9.54753705e-05 2.04578154e-02 1.90459825e-02 6.08261500e-04
 8.99520601e-05 4.05635888e-07 1.96706712e-01 3.39302671e-04
 1.30270053e-07 4.47016299e-01 8.90444517e-01 7.23250628e-01
 1.74837798e-01 1.68977858e-04 4.21205396e-03 6.25139833e-08
 5.94746552e-06 2.23003677e-03 2.16521556e-03 2.31984188e-03
 2.86520608e-02

Train Epoch: 10 [0/43 (0%)]	Train Loss: 0.018268
Train Epoch: 10 [10/43 (23%)]	Train Loss: 0.003028
Train Epoch: 10 [20/43 (47%)]	Train Loss: 0.045932
Train Epoch: 10 [30/43 (70%)]	Train Loss: 0.078819
Train Epoch: 10 [40/43 (93%)]	Train Loss: 0.005532

Train set: Average loss: 0.0155, Accuracy: 402/425 (95%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [1.69355873e-04 1.90292653e-02 3.85075845e-02 1.14777088e-02
 6.93012914e-03 2.41325561e-05 1.98538467e-01 4.69351333e-04
 2.77304352e-04 9.43294227e-01 9.81896043e-01 9.31911528e-01
 6.96760058e-01 1.36833078e-05 1.81858908e-04 2.00806608e-06
 7.89214508e-04 1.68675967e-02 3.01712006e-02 1.02560697e-02
 5.8986151

In [ ]:
# test
bs = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    
    targetlist, scorelist, predlist = test(epoch)
#     print('target',targetlist)
#     print('score',scorelist)
#     print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 
    
    TP = ((predlist == 1) & (targetlist == 1)).sum()
    TN = ((predlist == 0) & (targetlist == 0)).sum()
    FN = ((predlist == 0) & (targetlist == 1)).sum()
    FP = ((predlist == 1) & (targetlist == 0)).sum()

    print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
    print('TP+FP',TP+FP)
    p = TP / (TP + FP)
    print('precision',p)
    p = TP / (TP + FP)
    r = TP / (TP + FN)
    print('recall',r)
    F1 = 2 * r * p / (r + p)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print('F1',F1)
    print('acc',acc)
    AUC = roc_auc_score(targetlist, vote_score)
    print('AUC', AUC)

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        
#         print('vote_pred', vote_pred)
#         print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUC', AUC)
        
        
#         f = open('model_result/{modelname}.txt', 'a+')
#         f.write('precision, recall, F1, acc= \n')
#         f.writelines(str(p))
#         f.writelines('\n')
#         f.writelines(str(r))
#         f.writelines('\n')
#         f.writelines(str(F1))
#         f.writelines('\n')
#         f.writelines(str(acc))
#         f.writelines('\n')
#         f.close()
        
        
        vote_pred = np.zeros((1,testset.__len__()))
        vote_score = np.zeros(testset.__len__())
        print('vote_pred',vote_pred)
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open(f'model_result/test_{modelname}.txt', 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()

TP= 86 TN= 64 FN= 19 FP= 34
TP+FP 120
precision 0.7166666666666667
recall 0.819047619047619
F1 0.7644444444444445
acc 0.7389162561576355
AUC 0.8263362487852284
TP= 86 TN= 64 FN= 19 FP= 34
TP+FP 120
precision 0.7166666666666667
recall 0.819047619047619
F1 0.7644444444444445
acc 0.7389162561576355
AUC 0.8263362487852284
TP= 86 TN= 64 FN= 19 FP= 34
TP+FP 120
precision 0.7166666666666667
recall 0.819047619047619
F1 0.7644444444444445
acc 0.7389162561576355
AUC 0.8263362487852284
TP= 86 TN= 64 FN= 19 FP= 34
TP+FP 120
precision 0.7166666666666667
recall 0.819047619047619
F1 0.7644444444444445
acc 0.7389162561576355
AUC 0.8263362487852284
TP= 86 TN= 64 FN= 19 FP= 34
TP+FP 120
precision 0.7166666666666667
recall 0.819047619047619
F1 0.7644444444444445
acc 0.7389162561576355
AUC 0.8263362487852284
TP= 86 TN= 64 FN= 19 FP= 34
TP+FP 120
precision 0.7166666666666667
recall 0.819047619047619
F1 0.7644444444444445
acc 0.7389162561576355
AUC 0.8263362487852284
TP= 86 TN= 64 FN= 19 FP= 34
TP+FP 120
pr

## ResNet50

In [ ]:
### ResNet50
import torchvision.models as models
model = models.resnet50(pretrained=True)
modelname = 'ResNet50'

In [ ]:
# train
bs = 10
votenum = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    train(optimizer, epoch)
    
    targetlist, scorelist, predlist = val(epoch)
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        #torch.save(model.state_dict(), "model_backup/{}.pt".format(modelname))  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open('model_result/{}.txt'.format(modelname), 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()


Train Epoch: 1 [0/43 (0%)]	Train Loss: 0.885339
Train Epoch: 1 [10/43 (23%)]	Train Loss: 0.558124
Train Epoch: 1 [20/43 (47%)]	Train Loss: 0.222518
Train Epoch: 1 [30/43 (70%)]	Train Loss: 0.112035
Train Epoch: 1 [40/43 (93%)]	Train Loss: 0.019702

Train set: Average loss: 0.3494, Accuracy: 226/425 (53%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [1.13400383e-04 2.97591507e-01 7.97293335e-02 8.98024514e-02
 4.03693528e-04 8.49489006e-04 1.81826949e-01 4.21823144e-01
 6.01294711e-02 8.07129345e-06 5.78535728e-05 6.13427314e-04
 6.00466330e-04 2.50485122e-01 1.87171802e-01 2.48506808e-06
 1.56410999e-06 2.91835610e-02 4.39641080e-05 1.17536634e-01
 4.95704263e-01

Train Epoch: 4 [0/43 (0%)]	Train Loss: 0.027471
Train Epoch: 4 [10/43 (23%)]	Train Loss: 0.018163
Train Epoch: 4 [20/43 (47%)]	Train Loss: 0.052124
Train Epoch: 4 [30/43 (70%)]	Train Loss: 0.021611
Train Epoch: 4 [40/43 (93%)]	Train Loss: 0.074263

Train set: Average loss: 0.0361, Accuracy: 378/425 (89%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [1.03341844e-02 4.62577231e-02 5.88858826e-03 1.70899540e-01
 7.49352970e-04 1.47330463e-02 7.59265870e-02 4.54774946e-01
 1.43389702e-01 1.39166275e-03 5.35514235e-01 3.40759347e-04
 1.45974159e-01 9.45131201e-03 6.17146939e-02 2.83310465e-06
 6.90961178e-06 4.93806554e-03 1.23113859e-03 1.63981229e-01
 5.18880129e-01

Train Epoch: 7 [0/43 (0%)]	Train Loss: 0.000959
Train Epoch: 7 [10/43 (23%)]	Train Loss: 0.001939
Train Epoch: 7 [20/43 (47%)]	Train Loss: 0.044938
Train Epoch: 7 [30/43 (70%)]	Train Loss: 0.000536
Train Epoch: 7 [40/43 (93%)]	Train Loss: 0.002893

Train set: Average loss: 0.0108, Accuracy: 414/425 (97%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [2.29728787e-04 9.16091949e-02 4.55843285e-03 8.00031517e-03
 3.48232825e-05 4.85590535e-05 5.68517186e-02 2.13714577e-02
 2.03727061e-04 3.07242299e-05 6.39357185e-03 6.15058234e-05
 7.72200851e-03 8.62205168e-04 1.75194920e-03 1.04445621e-06
 1.56788167e-06 8.61515477e-03 2.32488019e-04 1.99371530e-03
 1.17841261e-02

Train Epoch: 10 [0/43 (0%)]	Train Loss: 0.002850
Train Epoch: 10 [10/43 (23%)]	Train Loss: 0.001118
Train Epoch: 10 [20/43 (47%)]	Train Loss: 0.000321
Train Epoch: 10 [30/43 (70%)]	Train Loss: 0.000229
Train Epoch: 10 [40/43 (93%)]	Train Loss: 0.023775

Train set: Average loss: 0.0086, Accuracy: 414/425 (97%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [1.65811682e-03 7.98827648e-01 3.96406651e-01 1.76025078e-01
 7.12350337e-03 4.15687310e-03 8.95766616e-01 1.60606876e-01
 1.89380981e-02 2.10812758e-03 5.37726358e-02 1.73355006e-02
 3.61889228e-02 8.56690407e-01 7.40965366e-01 1.85009048e-04
 4.51749598e-04 1.51044652e-01 3.28108814e-04 7.48709962e-03
 8.2168746

In [ ]:
# test
bs = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    
    targetlist, scorelist, predlist = test(epoch)
#     print('target',targetlist)
#     print('score',scorelist)
#     print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 
    
    TP = ((predlist == 1) & (targetlist == 1)).sum()
    TN = ((predlist == 0) & (targetlist == 0)).sum()
    FN = ((predlist == 0) & (targetlist == 1)).sum()
    FP = ((predlist == 1) & (targetlist == 0)).sum()

    print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
    print('TP+FP',TP+FP)
    p = TP / (TP + FP)
    print('precision',p)
    p = TP / (TP + FP)
    r = TP / (TP + FN)
    print('recall',r)
    F1 = 2 * r * p / (r + p)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print('F1',F1)
    print('acc',acc)
    AUC = roc_auc_score(targetlist, vote_score)
    print('AUC', AUC)

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        
#         print('vote_pred', vote_pred)
#         print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUC', AUC)
        
        
#         f = open('model_result/{modelname}.txt', 'a+')
#         f.write('precision, recall, F1, acc= \n')
#         f.writelines(str(p))
#         f.writelines('\n')
#         f.writelines(str(r))
#         f.writelines('\n')
#         f.writelines(str(F1))
#         f.writelines('\n')
#         f.writelines(str(acc))
#         f.writelines('\n')
#         f.close()
        
        
        vote_pred = np.zeros((1,testset.__len__()))
        vote_score = np.zeros(testset.__len__())
        print('vote_pred',vote_pred)
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open(f'model_result/test_{modelname}.txt', 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()

TP= 93 TN= 73 FN= 12 FP= 25
TP+FP 118
precision 0.788135593220339
recall 0.8857142857142857
F1 0.8340807174887892
acc 0.8177339901477833
AUC 0.8822157434402332
TP= 93 TN= 73 FN= 12 FP= 25
TP+FP 118
precision 0.788135593220339
recall 0.8857142857142857
F1 0.8340807174887892
acc 0.8177339901477833
AUC 0.8822157434402332
TP= 93 TN= 73 FN= 12 FP= 25
TP+FP 118
precision 0.788135593220339
recall 0.8857142857142857
F1 0.8340807174887892
acc 0.8177339901477833
AUC 0.8822157434402332
TP= 93 TN= 73 FN= 12 FP= 25
TP+FP 118
precision 0.788135593220339
recall 0.8857142857142857
F1 0.8340807174887892
acc 0.8177339901477833
AUC 0.8822157434402332
TP= 93 TN= 73 FN= 12 FP= 25
TP+FP 118
precision 0.788135593220339
recall 0.8857142857142857
F1 0.8340807174887892
acc 0.8177339901477833
AUC 0.8822157434402332
TP= 93 TN= 73 FN= 12 FP= 25
TP+FP 118
precision 0.788135593220339
recall 0.8857142857142857
F1 0.8340807174887892
acc 0.8177339901477833
AUC 0.8822157434402332
TP= 93 TN= 73 FN= 12 FP= 25
TP+FP 118
pr

## VGG16

In [ ]:
### VGGNet
import torchvision.models as models
model = models.vgg16(pretrained=True)
model = model
modelname = 'vgg16'

In [ ]:
# train
bs = 10
votenum = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(valset.__len__())
vote_score = np.zeros(valset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
#scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    train(optimizer, epoch)
    
    targetlist, scorelist, predlist = val(epoch)
    print('target',targetlist)
    print('score',scorelist)
    print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        vote_score = vote_score/votenum
        
        print('vote_pred', vote_pred)
        print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUCp', roc_auc_score(targetlist, vote_pred))
        print('AUC', AUC)
        
        
        
#         if epoch == total_epoch:
        #torch.save(model.state_dict(), "model_backup/{}.pt".format(modelname))  
        
        vote_pred = np.zeros(valset.__len__())
        vote_score = np.zeros(valset.__len__())
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open('model_result/{}.txt'.format(modelname), 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()


Train Epoch: 1 [0/43 (0%)]	Train Loss: 1.296838
Train Epoch: 1 [10/43 (23%)]	Train Loss: 0.050856
Train Epoch: 1 [20/43 (47%)]	Train Loss: 0.066426
Train Epoch: 1 [30/43 (70%)]	Train Loss: 0.041857
Train Epoch: 1 [40/43 (93%)]	Train Loss: 0.081799

Train set: Average loss: 0.1263, Accuracy: 253/425 (60%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [0.24109182 0.5731492  0.48328939 0.25816989 0.14369661 0.11580111
 0.67121387 0.38882363 0.1789974  0.41290295 0.16804112 0.33977953
 0.09643118 0.52563828 0.56042993 0.23557769 0.27367964 0.35927659
 0.37046161 0.45770612 0.50547349 0.55869263 0.59884667 0.56686121
 0.6372757  0.66577405 0.48304546 0.54902506 0.45341

Train Epoch: 5 [0/43 (0%)]	Train Loss: 0.005226
Train Epoch: 5 [10/43 (23%)]	Train Loss: 0.016584
Train Epoch: 5 [20/43 (47%)]	Train Loss: 0.040386
Train Epoch: 5 [30/43 (70%)]	Train Loss: 0.018138
Train Epoch: 5 [40/43 (93%)]	Train Loss: 0.049874

Train set: Average loss: 0.0334, Accuracy: 364/425 (86%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [0.02158675 0.98916316 0.94986153 0.0729586  0.02197046 0.01766023
 0.99076325 0.17613521 0.02542152 0.21223184 0.01170649 0.00991827
 0.03018623 0.1326911  0.09286632 0.03666371 0.03419546 0.13917257
 0.3857803  0.37282014 0.59321535 0.58335525 0.44932219 0.86114073
 0.7426275  0.76385462 0.75120676 0.66971731 0.22271

Train Epoch: 8 [30/43 (70%)]	Train Loss: 0.077455
Train Epoch: 8 [40/43 (93%)]	Train Loss: 0.035026

Train set: Average loss: 0.0253, Accuracy: 383/425 (90%)

target [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
score [8.06314964e-03 3.32289189e-01 1.19539864e-01 5.62741142e-03
 8.32336373e-04 1.32697542e-05 1.67309865e-01 4.36638966e-02
 8.31431476e-04 2.34438088e-02 8.27905815e-03 1.00936217e-03
 2.96104699e-02 5.78068011e-02 2.40110710e-01 1.42788077e-02
 2.06598155e-02 1.68019515e-02 1.52482629e-01 5.17271981e-02
 1.30967885e-01 2.46069819e-01 4.16326314e-01 8.79262626e-01
 1.83290362e-01 5.15935779e-01 6.99962020e-01 1.72674492e-01
 1.01581113e-02 9.78030730e-03 4.95355040

AUCp 0.771551724137931
AUC 0.8675287356321839

 The epoch is 10, average recall: 0.7931, average precision: 0.7541,average F1: 0.7731, average accuracy: 0.7712, average AUC: 0.8675


In [ ]:
# test
bs = 10
import warnings
warnings.filterwarnings('ignore')

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
vote_pred = np.zeros(testset.__len__())
vote_score = np.zeros(testset.__len__())

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scheduler = StepLR(optimizer, step_size=1)

total_epoch = 10
for epoch in range(1, total_epoch+1):
    
    targetlist, scorelist, predlist = test(epoch)
#     print('target',targetlist)
#     print('score',scorelist)
#     print('predict',predlist)
    vote_pred = vote_pred + predlist 
    vote_score = vote_score + scorelist 
    
    TP = ((predlist == 1) & (targetlist == 1)).sum()
    TN = ((predlist == 0) & (targetlist == 0)).sum()
    FN = ((predlist == 0) & (targetlist == 1)).sum()
    FP = ((predlist == 1) & (targetlist == 0)).sum()

    print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
    print('TP+FP',TP+FP)
    p = TP / (TP + FP)
    print('precision',p)
    p = TP / (TP + FP)
    r = TP / (TP + FN)
    print('recall',r)
    F1 = 2 * r * p / (r + p)
    acc = (TP + TN) / (TP + TN + FP + FN)
    print('F1',F1)
    print('acc',acc)
    AUC = roc_auc_score(targetlist, vote_score)
    print('AUC', AUC)

    if epoch % votenum == 0:
        
        # major vote
        vote_pred[vote_pred <= (votenum/2)] = 0
        vote_pred[vote_pred > (votenum/2)] = 1
        
#         print('vote_pred', vote_pred)
#         print('targetlist', targetlist)
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, vote_score)
        print('AUC', AUC)
        
        
#         f = open('model_result/{modelname}.txt', 'a+')
#         f.write('precision, recall, F1, acc= \n')
#         f.writelines(str(p))
#         f.writelines('\n')
#         f.writelines(str(r))
#         f.writelines('\n')
#         f.writelines(str(F1))
#         f.writelines('\n')
#         f.writelines(str(acc))
#         f.writelines('\n')
#         f.close()
        
        
        vote_pred = np.zeros((1,testset.__len__()))
        vote_score = np.zeros(testset.__len__())
        print('vote_pred',vote_pred)
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r, p, F1, acc, AUC))

        #f = open(f'model_result/test_{modelname}.txt', 'a+')
        #f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        #epoch, r, p, F1, acc, AUC))
        #f.close()

TP= 99 TN= 50 FN= 6 FP= 48
TP+FP 147
precision 0.673469387755102
recall 0.9428571428571428
F1 0.7857142857142857
acc 0.7339901477832512
AUC 0.8700680272108844
TP= 99 TN= 50 FN= 6 FP= 48
TP+FP 147
precision 0.673469387755102
recall 0.9428571428571428
F1 0.7857142857142857
acc 0.7339901477832512
AUC 0.8700680272108844
TP= 99 TN= 50 FN= 6 FP= 48
TP+FP 147
precision 0.673469387755102
recall 0.9428571428571428
F1 0.7857142857142857
acc 0.7339901477832512
AUC 0.8700680272108844
TP= 99 TN= 50 FN= 6 FP= 48
TP+FP 147
precision 0.673469387755102
recall 0.9428571428571428
F1 0.7857142857142857
acc 0.7339901477832512
AUC 0.8700680272108844
TP= 99 TN= 50 FN= 6 FP= 48
TP+FP 147
precision 0.673469387755102
recall 0.9428571428571428
F1 0.7857142857142857
acc 0.7339901477832512
AUC 0.8700680272108844
TP= 99 TN= 50 FN= 6 FP= 48
TP+FP 147
precision 0.673469387755102
recall 0.9428571428571428
F1 0.7857142857142857
acc 0.7339901477832512
AUC 0.8700680272108844
TP= 99 TN= 50 FN= 6 FP= 48
TP+FP 147
precision